上一篇笔记我们讲到了关联分析的基本概念和应用场景，以及挖掘数据集中关联规则的**Apriori算法**，通过具体代码实现了一个Apriori算法，在上一篇笔记的最后提到Apriori算法的效率并不高，因此本章就深入一个优化了的关联规则算法FP-growth。


FP-growth算法是伊利罗伊香槟分校的韩嘉炜教授于2004年[1]提出的，它是为了解决Apriori算法每次增加频繁项集的大小都要遍历整个数据库的缺点，特别是当数据集很大时，该算法执行速度要快于Apriori算法两个数量级。FP-growth算法的任务是将数据集存储在一个特定的称为FP树的结构之后发现频繁项集或者频繁项对，虽然它能够高效地发现频繁项集，但是不能用来发现关联规则，也就是只优化了Apriori算法两个功能中的前一个功能。


FP-growth算法将数据存储在一个称为FP树的紧凑数据结构中，它与计算机科学中的其他树的结构类似，但是它通过链接来链接相似元素，被连起来的元素可以看做一个链表。

![包含相似节点链接的FP树](./dsIpynbPic/FP_Tree_chpten.png)


  FP-growth算法只需要对数据集进行两次扫描，所以即使数据集很大时也不会花费太多的时间在扫描数据上，它发现频繁项集的基本过程如下：
    1）构建FP树
    2）从FP树中挖掘频繁项集




### 创建FP Tree的数据结构

对每一频繁项，都要创建一颗条件FP树，将上面的条件模式基作为输入，通过相同的建树方法来构建这些条件树，然后递归地发现频繁项、发现条件模式基，以及发现另外的条件树。举个例子，假定为频繁项t创建一个条件FP树，然后对{t,y}，{t,x}、...重复该过程。

构建相关类的代码如下：

In [4]:
class treeNode:  #FP Tree的树节点
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur  #计数
        self.parent = parentNode     #父节点
        self.nodeLink = None  #横向链
        self.children = {}   #子节点
    
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1): #将树以文本形式显示
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)


创建 FP 树的函数：

In [5]:
def createTree(dataSet, minSup=1): #从数据集创建FP Tree
    headerTable = {}
    #两次遍历数据集
    for trans in dataSet:#第一次遍历：统计每个元素项出现次数
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    klst=[j for j in  headerTable.keys()]
    for k in klst:  #移除不满足最小支持度的元素项
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0: return None, None  #i如果没有元素项满足要求，退出
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] 
    retTree = treeNode('Null Set', 1, None) #建立树的节点
    for tranSet, count in dataSet.items():  
        localD = {}
        for item in tranSet:  
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0: #对剩下的元素项迭代，调用updateTree
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable 

def updateTree(items, inTree, headerTable, count): #增长更新树
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count) 
    else: #加入i[0]
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)
        
def updateHeader(nodeToTest, targetNode):  
    while (nodeToTest.nodeLink != None):   
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

**抽取条件模式基**(conditional pattern base)以及递归查找频繁项集。

In [6]:
#发现以给定元素项结尾的所有路径
def ascendTree(leafNode, prefixPath): #迭代上溯整棵树
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
    
def findPrefixPath(basePat, treeNode): 
    #返回basePat对于的所有前缀和计数,treeNode是其对应的第一个节点
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

def mineTree(inTree, headerTable, minSup, preFix, freqItemList): #递归查找频繁项集的函数
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]#排序头指针
    for basePat in bigL:  #从头指针表的底端开始
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        #从条件模式基构建FP树
        myCondTree, myHead = createTree(condPattBases, minSup)
        
        if myHead != None: #挖掘条件FP树
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)


创建一个虚拟的数据集进行测试：

In [7]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

#测试数据
simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

#调用

initSet = createInitSet(simpDat)
myFPtree, myHeaderTab = createTree(initSet,3)
myFPtree.disp()
myFreqList = []
mineTree(myFPtree, myHeaderTab, 3, set([]), myFreqList)

   Null Set   1
     z   5
       r   1
       x   3
         y   3
           s   2
             t   2
           r   1
             t   1
     x   1
       s   1
         r   1


关联分析还有其他的算法，以后自己肯定还会进行更深入的学习，这两篇文章分别记录了**Apriori算法**和**FP growth算法**，这两个算法是很基础的，并且我们从FP growth算法对Apriori算法的改进中也应该体会这种优化的思维。这在自己建模以及使用经典模型中是很重要的。

本系列笔记到这里也告一段落了，之后的学习还会继续补充**数据分析**和**机器学习**的内容，但估计不会每天更新。